In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.ensemble import IsolationForest
import optuna

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# import warnings
# warnings.simplefilter(action='ignore', category=UserWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Data import**

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv", low_memory=False)#, nrows=10000)
train["date_time"] = pd.to_datetime(train["date_time"], format="%Y-%m-%d %H:%M:%S")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv", low_memory=False)
test["date_time"] = pd.to_datetime(test["date_time"], format="%Y-%m-%d %H:%M:%S")
train.info(memory_usage="deep")

In [ ]:
test.info(memory_usage="deep")

In [ ]:
train.head(10)

# **EDA**

In [ ]:
targets = ["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"]
target_names = ["Carbon monoxide", "Benzene", "Nitrogen oxides"]

The idea of SMC feature below was taken from this [notebook](https://www.kaggle.com/junhyeok99/automl-pycaret).

Taking into account temperature changes was suggested by [@lukaszborecki](https://www.kaggle.com/lukaszborecki) [here](https://www.kaggle.com/c/tabular-playground-series-jul-2021/discussion/250931#1380107).

In [ ]:
def add_new_plot_features(df):
    """
    Adds new features to a given dataset for plotting
    """
    df["month"] = df["date_time"].dt.month
    df["day_of_week"] = df["date_time"].dt.dayofweek
    df["day_of_year"] = df["date_time"].dt.dayofyear
    df["hour"] = df["date_time"].dt.hour
    df["quarter"] = df["date_time"].dt.quarter
    df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype("int")
#     df["is_winter"] = df["month"].isin([1, 2, 12])
#     df["is_sprint"] = df["month"].isin([3, 4, 5])
#     df["is_summer"] = df["month"].isin([6, 7, 8])
#     df["is_autumn"] = df["month"].isin([9, 10, 11])
    df["working_hours"] =  df["hour"].isin(np.arange(8, 21, 1)).astype("int")
    df["is_weekend"] = (df["date_time"].dt.dayofweek >= 5).astype("int")
    return df

def add_new_ml_features(df, i=3): # i=3 is for heatmap plot
    """
    Adds new features to a given dataset for training
    """

    df["hour"] = df["date_time"].dt.hour
    df["working_hours"] =  df["hour"].isin(np.arange(8, 21, 1)).astype("int")
    df["maximum_hours"] =  df["hour"].isin([8, 9, 17, 18, 19, 20]).astype("int")
    
    # Marking weekends because they usually have lower target values
    df["is_weekend"] = (df["date_time"].dt.dayofweek >= 5).astype("int")
    df["SMC"] = (df["absolute_humidity"] * 100) / df["relative_humidity"]
    
    # A list of features to generate shifted and lagged values
    shift_features = [["SMC", "absolute_humidity", "deg_C",
                      "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"],
                      ["SMC", "absolute_humidity", "target_carbon_monoxide_preds",
                      "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"],
                      ["SMC", "absolute_humidity", "target_carbon_monoxide_preds", "target_benzene_preds",
                      "sensor_1", "sensor_2", "sensor_3", "sensor_5"],
                      # Features for heatmap plot
                      ["SMC", "absolute_humidity", "deg_C",
                      "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"]]
    
    # Amounts of hour shifts and lags
    shifts = [1, 2, 3, 4, 5, 6, 12, 24]
    
    for feature in shift_features[i]:
        for shift in shifts:
            df[feature+"-"+str(shift)+"abs_shfit"] = df[feature] - df[feature].shift(periods=shift, fill_value=0)
            df[feature+"+"+str(shift)+"abs_shfit"] = df[feature] - df[feature].shift(periods=-shift, fill_value=0)

    # Replacing infinity values as a result of devision by zero at the end of a dataset
    df.replace(to_replace=np.inf, value=0, inplace=True)
    
    return df.drop("hour", axis=1)


In [ ]:
train_copy = train.copy()
test_copy = test.copy()
train = add_new_plot_features(train)
test = add_new_plot_features(test)

The datasets have timestamps. Let's compare which dates are in each dataset.

In [ ]:
# Plot dataframe
df = pd.concat([train["date_time"], test["date_time"]], axis=0).reset_index(drop=True)

fig, ax = plt.subplots(figsize=(16, 1.5))
bar1 =  ax.barh(0, 7111+2247, color="salmon", height=0.2)
bar2 =  ax.barh(0, 7111, color="teal", height=0.2)
ax.set_title("Train and test datasets size comparison", fontsize=20, pad=5)
ax.bar_label(bar1, ["Test dataset"], label_type="edge", padding=-170,
             fontsize=20, color="white", weight="bold")
ax.bar_label(bar2, ["Train dataset"], label_type="center",
             fontsize=20, color="white", weight="bold")
ax.set_xticks([0, 7111, 7111+2247])
ax.set_xticklabels(["2010-03-10", "2011-01-01", "2011-04-04"])
ax.set_yticks([])
plt.show();

## Target plots

The datasets also have three target columns that the model have to predict. Let's see how each target is changing in time.

In [ ]:
fig, axs = plt.subplots(figsize=(16, 18), ncols=1, nrows=3, sharex=False)

plt.subplots_adjust(hspace = 0.3)

colors = ["palevioletred", "deepskyblue", "mediumseagreen"]

for i in [0, 1, 2]:
    axs[i].plot(train["date_time"], train[targets[i]], color=colors[i])
    axs[i].set_title(f"{target_names[i]} (target #{i+1}) levels across time", fontsize=20, pad=5)
    axs[i].set_ylabel(f"{target_names[i]} level", fontsize=14, labelpad=5)
    axs[i].set_xlabel("Date", fontsize=14, labelpad=5)
    axs[i].grid(axis="both")

plt.show();

Let's see mean target values per day of year.

In [ ]:
# Dataframe copy excluding the last row which is the only one representing January
df = train.drop([7110], axis=0).copy()
df["day"] = df["date_time"].dt.dayofyear
df["weekday"] = df["date_time"].dt.dayofweek

colors = ["palevioletred", "deepskyblue", "mediumseagreen"]

# An array of number of days of year (i.e. from 1 to 365) which are mondays to mark week starts
mondays = df.loc[df["weekday"] == 0]["day"].value_counts(sort=False).index
# An array of number of weeks of year to be used as label ticks
weeks = df["date_time"].dt.isocalendar().week.unique()[1:]

fig, axs = plt.subplots(figsize=(16, 18), ncols=1, nrows=3, sharex=False)

plt.subplots_adjust(hspace = 0.3)

for i in [0, 1, 2]:
    axs[i].plot(df.groupby("day")[targets[i]].mean().index,
                df.groupby("day")[targets[i]].mean().values, color=colors[i])
    axs[i].set_title(f"{target_names[i]} (target #{i+1}) mean levels across time", fontsize=20, pad=5)
    axs[i].set_ylabel(f"{target_names[i]} level", fontsize=14, labelpad=5)
    axs[i].set_xlabel("Week starts", fontsize=14, labelpad=5)
    axs[i].set_xticks(mondays)
    axs[i].set_xticklabels(weeks)
    axs[i].grid(axis="both")

plt.show();

As you can see, all target values usually go down at the end of each week (i.e. during weekends). 

Let's check targets distribution along each month.

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=5, figsize=(16, 20))
plt.subplots_adjust(hspace = 0.3)
fig.suptitle(target_names[0], fontsize=20)

i=3
for r in np.arange(5):
    for c in [0, 1]:
        axs[r, c].plot(train.loc[train["month"]==i, targets[0]], color="steelblue")
        axs[r, c].set_title(f"Month #{i}", fontsize=15)
        axs[r, c].legend(fontsize=13)
        i+=1

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=5, figsize=(16, 20))
plt.subplots_adjust(hspace = 0.3)
fig.suptitle(target_names[1], fontsize=20)

i=3
for r in np.arange(5):
    for c in [0, 1]:
        axs[r, c].plot(train.loc[train["month"]==i, targets[1]], color="palevioletred")
        axs[r, c].set_title(f"Month #{i}", fontsize=15)
        axs[r, c].legend(fontsize=13)
        i+=1

There are some near zero flat areas at 4th, 6th, 8th, 12th month plots. Need to figure out what is so special about these days. It also may be a garbage data which sould be deleted before machine learning.

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=5, figsize=(16, 20))
plt.set_cmap("Set2")
plt.subplots_adjust(hspace = 0.3)
fig.suptitle(target_names[2], fontsize=20)

i=3
for r in np.arange(5):
    for c in [0, 1]:
        axs[r, c].plot(train.loc[train["month"]==i, targets[2]], color="goldenrod")
        axs[r, c].set_title(f"Month #{i}", fontsize=15)
        axs[r, c].legend(fontsize=13)
        i+=1

Let's check each target value distribution.

In [ ]:
fig, axs = plt.subplots(figsize=(15, 6), ncols=3, nrows=1, sharey=False)

fig.suptitle("Target values distribution", fontsize=20)

colors = ["mediumorchid", "lightseagreen", "cornflowerblue"]

for i in [0, 1, 2]:
    axs[i].hist(train[targets[i]], bins=60, edgecolor="black", color=colors[i])
    axs[i].set_title(f"{target_names[i]} (target #{i+1})", fontsize=15, pad=5)
    axs[i].set_ylabel("Amount of values", fontsize=13, labelpad=5)
    axs[i].set_xlabel(f"{target_names[i]} level", fontsize=13, labelpad=5)
    axs[i].grid(axis="y")

plt.show();

Let's check how each target value chenges depending on the time of day, day of week, and month.

In [ ]:
fig, axs = plt.subplots(figsize=(16, 18), ncols=1, nrows=3, sharex=False)

plt.subplots_adjust(hspace = 0.3)
width=0.35
x = train.groupby("hour")["target_carbon_monoxide"].mean().index

for i in np.arange(3):
    bars1 = axs[i].bar(x-width/2, train.groupby("hour")[targets[i]].mean(),
                        width=width, edgecolor="black", label="Mean", color="cornflowerblue")
    bars2 = axs[i].bar(x+width/2, train.groupby("hour")[targets[i]].median(),
                        width=width, edgecolor="black", label="Median", color="palevioletred")
    axs[i].set_title(f"{target_names[i]} (target #{i+1})", fontsize=15, pad=10)
    axs[i].set_ylabel("Target value", fontsize=13, labelpad=5)
    axs[i].set_xlabel("Day hours", fontsize=13, labelpad=5)
    axs[i].set_xticks(x)
    axs[i].grid(axis="y")
    axs[i].legend(fontsize=13)


In [ ]:
# Dataframe copy excluding the last row which is the only one representing January
df = train.drop([7110], axis=0).copy()

fig, axs = plt.subplots(figsize=(16, 19), ncols=2, nrows=3, sharex=False,
                        gridspec_kw={'width_ratios': [1, 1.5]})

fig.suptitle("Target values distribution per month and day of week", fontsize=20)

plt.subplots_adjust(hspace = 0.25)
width=0.35
x = df.groupby("day_of_week")["target_carbon_monoxide"].mean().index + 1

for i in np.arange(3):
    bars1 = axs[i, 0].bar(x-width/2, df.groupby("day_of_week")[targets[i]].mean(),
                        width=width, edgecolor="black", label="Mean", color="salmon")
    bars2 = axs[i, 0].bar(x+width/2, df.groupby("day_of_week")[targets[i]].median(),
                        width=width, edgecolor="black", label="Median", color="teal")
    axs[i, 0].set_title(f"{target_names[i]} (target #{i+1})", fontsize=15, pad=10)
    axs[i, 0].set_ylabel("Target value", fontsize=13, labelpad=5)
    axs[i, 0].set_xlabel("Day of week", fontsize=13, labelpad=5)
    axs[i, 0].set_xticks(x)
    axs[i, 0].grid(axis="y")
    axs[i, 0].legend(fontsize=13)

x = df.groupby("month")["target_carbon_monoxide"].mean().index
for i in np.arange(3):
    bars1 = axs[i, 1].bar(x-width/2, df.groupby("month")[targets[i]].mean(),
                        width=width, edgecolor="black", label="Mean", color="salmon")
    bars2 = axs[i, 1].bar(x+width/2, df.groupby("month")[targets[i]].median(),
                        width=width, edgecolor="black", label="Median", color="teal")
    axs[i, 1].set_title(f"{target_names[i]} (target #{i+1})", fontsize=15, pad=10)
    axs[i, 1].set_ylabel("Target value", fontsize=13, labelpad=5)
    axs[i, 1].set_xlabel("Month", fontsize=13, labelpad=5)
    axs[i, 1].set_xticks(x)
    axs[i, 1].grid(axis="y")
    axs[i, 1].legend(fontsize=13)

In [ ]:
# Day hours which will be used for plotting data
hours = [0, 5, 8, 14, 19]
# Dataframe copy excluding the last row which is the only one representing January
df = train.loc[train["hour"].isin(hours)].drop([7110], axis=0).copy()

fig, axs = plt.subplots(figsize=(16, 18), ncols=2, nrows=3, sharex=False,
                        gridspec_kw={'width_ratios': [1, 1.5]})

fig.suptitle("Target values distribution per month and day of week at given hours", fontsize=20)

plt.subplots_adjust(hspace = 0.3)
width=0.15
x = np.sort(df["day_of_week"].unique()) + 1

for i in np.arange(3):
    bars1 = axs[i, 0].bar(x-width*2, df.loc[df["hour"] == 0].groupby("day_of_week")[targets[i]].mean(),
                        width=width, edgecolor="black", label="00:00", color="salmon")
    bars2 = axs[i, 0].bar(x-width, df.loc[df["hour"] == 5].groupby("day_of_week")[targets[i]].mean(),
                        width=width, edgecolor="black", label="05:00", color="sandybrown")
    bars3 = axs[i, 0].bar(x, df.loc[df["hour"] == 8].groupby("day_of_week")[targets[i]].mean(),
                        width=width, edgecolor="black", label="08:00", color="teal")
    bars4 = axs[i, 0].bar(x+width, df.loc[df["hour"] == 14].groupby("day_of_week")[targets[i]].mean(),
                        width=width, edgecolor="black", label="14:00", color="palevioletred")
    bars5 = axs[i, 0].bar(x+width*2, df.loc[df["hour"] == 19].groupby("day_of_week")[targets[i]].mean(),
                        width=width, edgecolor="black", label="19:00", color="mediumslateblue")
    axs[i, 0].set_title(f"{target_names[i]} (target #{i+1})", fontsize=15, pad=10)
    axs[i, 0].set_ylabel("Target value", fontsize=13, labelpad=5)
    axs[i, 0].set_xlabel("Day of week", fontsize=13, labelpad=5)
    axs[i, 0].set_xticks(x)
    axs[i, 0].grid(axis="y")
    axs[i, 0].legend(fontsize=10)

x = df["month"].unique()
for i in np.arange(3):
    bars1 = axs[i, 1].bar(x-width*2, df.loc[df["hour"] == 0].groupby("month")[targets[i]].mean(),
                        width=width, edgecolor="black", label="00:00", color="salmon")
    bars2 = axs[i, 1].bar(x-width, df.loc[df["hour"] == 5].groupby("month")[targets[i]].mean(),
                        width=width, edgecolor="black", label="05:00", color="sandybrown")
    bars3 = axs[i, 1].bar(x, df.loc[df["hour"] == 8].groupby("month")[targets[i]].mean(),
                        width=width, edgecolor="black", label="08:00", color="teal")
    bars4 = axs[i, 1].bar(x+width, df.loc[df["hour"] == 14].groupby("month")[targets[i]].mean(),
                        width=width, edgecolor="black", label="14:00", color="palevioletred")
    bars5 = axs[i, 1].bar(x+width*2, df.loc[df["hour"] == 19].groupby("month")[targets[i]].mean(),
                        width=width, edgecolor="black", label="19:00", color="mediumslateblue")
    axs[i, 1].set_title(f"{target_names[i]} (target #{i+1})", fontsize=15, pad=10)
    axs[i, 1].set_ylabel("Target value", fontsize=13, labelpad=5)
    axs[i, 1].set_xlabel("Month", fontsize=13, labelpad=5)
    axs[i, 1].set_xticks(x)
    axs[i, 1].grid(axis="y")
    axs[i, 1].legend(fontsize=10)

## Feature plots

In [ ]:
# Lists of feature names to be used for plots below
all_features = ["deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"]
all_feature_names = ["Temperature (deg. C)", "Relative humidity", "Absolute humidity", "Sensor 1", "Sensor_2", "Sensor 3", "Sensor 4", "Sensor 5"]

weather_features = ["deg_C", "relative_humidity", "absolute_humidity"]
weather_feature_names = ["Temperature (deg. C)", "Relative humidity", "Absolute humidity"]

sensor_features = ["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"]
sensor_feature_names = ["Sensor 1", "Sensor_2", "Sensor 3", "Sensor 4", "Sensor 5"]

Let's compare our train and test feature data.

In [ ]:
fig, axs = plt.subplots(figsize=(16, 30), ncols=1, nrows=8, sharex=False)

plt.subplots_adjust(hspace = 0.4)

colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

for i in np.arange(8):
    legend_lines = [Line2D([0], [0], color=colors[i], lw=10),
                    Line2D([0], [0], color="black", lw=10)]
    axs[i].plot(train["date_time"], train[all_features[i]], color=colors[i], label="Train data")
    axs[i].plot(test["date_time"], test[all_features[i]], color="black", label="Test data")
    axs[i].set_title(f"{all_feature_names[i]} levels across time", fontsize=20, pad=5)
    axs[i].set_ylabel(f"{all_feature_names[i]} level", fontsize=14, labelpad=5)
    axs[i].set_xlabel("Date", fontsize=14, labelpad=5)
    axs[i].legend(legend_lines, ["Train data", "Test data"], fontsize=12, loc=1)
    axs[i].grid(axis="both")

In [ ]:
# Plot dataframe creation
df = pd.concat([train_copy, test_copy], axis=0)
df.reset_index(drop=True, inplace=True)
df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype("int")
df["day_of_year"] = df["date_time"].dt.dayofyear

fig, axs = plt.subplots(figsize=(16, 18), ncols=2, nrows=3, sharex=False)

plt.subplots_adjust(hspace = 0.4)

colors = ["palevioletred", "deepskyblue", "mediumseagreen"]

for i in [0, 1, 2]:
    # New year days start from 7110th row
    data = df.iloc[:7110].groupby("day_of_year")[weather_features[i]].mean()
    axs[i, 0].plot(data.index, data.values, color=colors[i], label="Train data")
    data = df.iloc[7110:].groupby("day_of_year")[weather_features[i]].mean()
    axs[i, 0].plot(data.index, data.values, color="black", alpha=0.7, label="Test data")
    axs[i, 0].set_title(f"Mean dayly {weather_feature_names[i]} levels", fontsize=20, pad=5)
    axs[i, 0].set_ylabel(f"{weather_feature_names[i]} level", fontsize=14, labelpad=5)
    axs[i, 0].set_xlabel("Day of year", fontsize=14, labelpad=5)
    axs[i, 0].grid(axis="both")
    axs[i, 0].legend(fontsize=12)


for i in [0, 1, 2]:
    # New year weeks start from 7159th row. 
    # Because of Jan 1st and 2nd from the test dataset are counted as 52nd week of 2010,
    # the colored plotline contains some test data. 
    data = df.iloc[:7159].groupby("week_of_year")[weather_features[i]].mean()
    axs[i, 1].plot(data.index, data.values, color=colors[i], label="Train data")
    data = df.iloc[7159:].groupby("week_of_year")[weather_features[i]].mean()
    axs[i, 1].plot(data.index, data.values, color="black", alpha=0.7, label="Test data")
    axs[i, 1].set_title(f"Mean weekly {weather_feature_names[i]} levels", fontsize=20, pad=5)
    axs[i, 1].set_ylabel(f"{weather_feature_names[i]} level", fontsize=14, labelpad=5)
    axs[i, 1].set_xlabel("Week of year", fontsize=14, labelpad=5)
    axs[i, 1].grid(axis="both")
    axs[i, 1].legend(fontsize=12)

plt.show();

In [ ]:
# Plot dataframe creation
df = pd.concat([train_copy, test_copy], axis=0)
df.reset_index(drop=True, inplace=True)
df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype("int")
df["day_of_year"] = df["date_time"].dt.dayofyear

fig, axs = plt.subplots(figsize=(16, 30), ncols=2, nrows=5, sharex=False)

plt.subplots_adjust(hspace = 0.4)

colors = ["palevioletred", "deepskyblue", "mediumseagreen", "goldenrod", "indianred"]

for i in np.arange(5):
    data = df.iloc[:7110].groupby("day_of_year")[sensor_features[i]].mean()
    axs[i, 0].plot(data.index, data.values, color=colors[i], label="Train data")
    data = df.iloc[7110:].groupby("day_of_year")[sensor_features[i]].mean()
    axs[i, 0].plot(data.index, data.values, color="black", alpha=0.7, label="Test data")
    axs[i, 0].set_title(f"Mean dayly {sensor_feature_names[i]} levels", fontsize=20, pad=5)
    axs[i, 0].set_ylabel(f"{sensor_feature_names[i]} level", fontsize=14, labelpad=5)
    axs[i, 0].set_xlabel("Day of year", fontsize=14, labelpad=5)
    axs[i, 0].grid(axis="both")
    axs[i, 0].legend(fontsize=12)


for i in np.arange(5):
    data = df.iloc[:7159].groupby("week_of_year")[sensor_features[i]].mean()
    axs[i, 1].plot(data.index, data.values, color=colors[i], label="Train data")
    data = df.iloc[7159:].groupby("week_of_year")[sensor_features[i]].mean()
    axs[i, 1].plot(data.index, data.values, color="black", alpha=0.7, label="Test data")
    axs[i, 1].set_title(f"Mean dayly {sensor_feature_names[i]} levels", fontsize=20, pad=5)
    axs[i, 1].set_ylabel(f"{sensor_feature_names[i]} level", fontsize=14, labelpad=5)
    axs[i, 1].set_xlabel("Week of year", fontsize=14, labelpad=5)
    axs[i, 1].grid(axis="both")
    axs[i, 1].legend(fontsize=12)

plt.show();

Let's check feature correlation.

In [ ]:
# Plot dataframe
df = train_copy.copy()
df = pd.concat([df[targets], df.drop(targets, axis=1)], axis=1).corr().round(2)

# Mask to hide upper-right part of plot as it is a duplicate
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(12,12))
ax = sns.heatmap(df, annot=True, mask=mask, cmap="RdBu", linewidth=1,
                 annot_kws={"weight": "bold", "fontsize":13})
ax.set_title("Original dataset correlation heatmap", fontsize=17)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor", weight="bold")
plt.setp(ax.get_yticklabels(), weight="bold")
plt.show();

In [ ]:
# Plot dataframe
df = add_new_ml_features(train_copy.copy())
df = pd.concat([df[targets], df.drop(targets, axis=1)], axis=1).corr().round(2)

# Mask to hide upper-right part of plot as it is a duplicate
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(16,12))
ax = sns.heatmap(df, annot=False, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize": 9})
ax.set_title("Original and engineered features correlation heatmap", fontsize=17)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor", weight="normal")
plt.setp(ax.get_yticklabels(), weight="normal")
plt.show();

# **Models training**

The datetime to int conversion shown below was found in this [notebook](https://www.kaggle.com/jarupula/eda-rf-model-tps-july-21).

In [ ]:
def prepare_dataset(train_copy, test_copy, i):
    """
    Preprocesses given train and test datasets and returns X, y, X_test 
    """

    X = add_new_ml_features(train_copy.copy(), i)

    # Dropping the last row which is 2011-01-01 00:00:00
    if X.index[-1] == 7110:
        X.drop([7110], axis=0, inplace=True)

    # Resetting dataframe index
    X.reset_index(drop=True, inplace=True)
    # Adding 168 last train set rows to the head of test set in order to get shifting feature values
    X_test_temp = pd.concat([train_copy.iloc[-169:-1].drop(["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"], axis=1), test_copy], axis=0)
    X_test_temp.reset_index(inplace=True, drop=True)
    X_test = add_new_ml_features(X_test_temp.copy(), i)
    # Deleting added train set rows
    X_test.drop(X_test.loc[:167].index, axis=0, inplace=True)
    X_test.reset_index(inplace=True, drop=True)

    y = np.log1p(X[["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"]])
    X.drop(["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"], axis=1, inplace=True)
    
    X['date_time'] = X['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9
    X_test['date_time'] = X_test['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9

#     display(X.head())
#     display(X_test.head())
#     display(y.head())

    return X, X_test, y

In [ ]:
# Sets of base models hyperparameters optimized by Optuna for each target with various depth
cb_params = [
                [{'iterations': 1000,
                  'learning_rate': 0.03298118997883894,
                  'l2_leaf_reg': 13.866168815111331,
                  'bagging_temperature': 3.598456367521501,
                  'random_strength': 1.1810169295356476,
                  'depth': 16,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Gradient',
                  'max_leaves': 64},
                 {'iterations': 1000,
                  'learning_rate': 0.03298118997883894,
                  'l2_leaf_reg': 13.866168815111331,
                  'bagging_temperature': 3.598456367521501,
                  'random_strength': 1.1810169295356476,
                  'depth': 15,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Gradient',
                  'max_leaves': 64},
                 {'iterations': 1000,
                  'learning_rate': 0.03298118997883894,
                  'l2_leaf_reg': 13.866168815111331,
                  'bagging_temperature': 3.598456367521501,
                  'random_strength': 1.1810169295356476,
                  'depth': 14,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Gradient',
                  'max_leaves': 64},
                 {'iterations': 20000,
                  'learning_rate': 0.023621545862016827,
                  'l2_leaf_reg': 19.030604497176707,
                  'bagging_temperature': 1.6654461295318517,
                  'random_strength': 1.3014359165957727,
                  'depth': 4,
                  'grow_policy': 'SymmetricTree',
                  'leaf_estimation_method': 'Gradient'},
                 {'iterations': 16427,
                  'learning_rate': 0.011270680049200347,
                  'l2_leaf_reg': 2.1567741919720036,
                  'bagging_temperature': 2.179300367880761,
                  'random_strength': 1.4991607805892369,
                  'depth': 4,
                  'grow_policy': 'Depthwise',
                  'leaf_estimation_method': 'Gradient'}],
    
    
                [{'iterations': 1000,
                  'learning_rate': 0.02721102734588763,
                  'l2_leaf_reg': 17.373548837368403,
                  'bagging_temperature': 5.321778377898784,
                  'random_strength': 1.442010850601074,
                  'depth': 16,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Newton',
                  'max_leaves': 64},
                 {'iterations': 1000,
                  'learning_rate': 0.02721102734588763,
                  'l2_leaf_reg': 17.373548837368403,
                  'bagging_temperature': 5.321778377898784,
                  'random_strength': 1.442010850601074,
                  'depth': 15,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Newton',
                  'max_leaves': 64},
                 {'iterations': 1000,
                  'learning_rate': 0.02721102734588763,
                  'l2_leaf_reg': 17.373548837368403,
                  'bagging_temperature': 5.321778377898784,
                  'random_strength': 1.442010850601074,
                  'depth': 14,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Newton',
                  'max_leaves': 64},
                 {'iterations': 10035,
                  'learning_rate': 0.010293242258606604,
                  'l2_leaf_reg': 1.00856046822379,
                  'bagging_temperature': 2.0084905523023475,
                  'random_strength': 1.0989704542465142,
                  'depth': 4,
                  'grow_policy': 'SymmetricTree',
                  'leaf_estimation_method': 'Newton'},
                 {'iterations': 7686,
                  'learning_rate': 0.01018369258714962,
                  'l2_leaf_reg': 16.303503907195584,
                  'bagging_temperature': 6.0525965608860615,
                  'random_strength': 1.7916045776213503,
                  'depth': 4,
                  'grow_policy': 'Depthwise',
                  'leaf_estimation_method': 'Newton'}],
    
    
                [{'iterations': 1000,
                  'learning_rate': 0.04162551441945943,
                  'l2_leaf_reg': 0.6161561831774424,
                  'bagging_temperature': 8.70578164844492,
                  'random_strength': 1.033027101582271,
                  'depth': 16,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Gradient',
                  'max_leaves': 64},
                 {'iterations': 1000,
                  'learning_rate': 0.04162551441945943,
                  'l2_leaf_reg': 0.6161561831774424,
                  'bagging_temperature': 8.70578164844492,
                  'random_strength': 1.033027101582271,
                  'depth': 15,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Gradient',
                  'max_leaves': 64},
                 {'iterations': 1000,
                  'learning_rate': 0.04162551441945943,
                  'l2_leaf_reg': 0.6161561831774424,
                  'bagging_temperature': 8.70578164844492,
                  'random_strength': 1.033027101582271,
                  'depth': 14,
                  'grow_policy': 'Lossguide',
                  'leaf_estimation_method': 'Gradient',
                  'max_leaves': 64},
                 {'iterations': 8859,
                  'learning_rate': 0.049661025418408236,
                  'l2_leaf_reg': 19.999333247464577,
                  'bagging_temperature': 9.584826983465707,
                  'random_strength': 1.8532006283239564,
                  'depth': 4,
                  'grow_policy': 'SymmetricTree',
                  'leaf_estimation_method': 'Newton'},
                 {'iterations': 19895,
                  'learning_rate': 0.010495629145711569,
                  'l2_leaf_reg': 18.350392839340135,
                  'bagging_temperature': 2.2661957751297495,
                  'random_strength': 1.449631439165256,
                  'depth': 4,
                  'grow_policy': 'Depthwise',
                  'leaf_estimation_method': 'Newton'}]
            ]

# Sets of blender models hyperparameters optimized by Optuna for each target
xgb_params = [{'n_estimators': 78,
               'learning_rate': 0.07527947300777532,
               'max_depth': 2,
               'booster': 'gbtree',
               'tree_method': 'auto',
               'reg_lambda': 0.6378105258790215,
               'reg_alpha': 0.0220722729914309,
               'random_state': 42,
               'n_jobs': 4},
              {'n_estimators': 333,
               'learning_rate': 0.025432541067690877,
               'max_depth': 2,
               'booster': 'gbtree',
               'tree_method': 'auto',
               'reg_lambda': 0.21247269011384323,
               'reg_alpha': 0.3029339596513587,
               'random_state': 42,
               'n_jobs': 4},
              {'n_estimators': 140,
               'learning_rate': 0.0633431641773825,
               'max_depth': 2,
               'booster': 'gbtree',
               'tree_method': 'auto',
               'reg_lambda': 0.1995327745605569,
               'reg_alpha': 0.7236251161640519,
               'random_state': 42,
               'n_jobs': 4}]

In [ ]:
def get_oof_rmsle(X, y, params, months):
    """
    Splits given train dataset then trains a model with given parameters on a train set and returns valid set RMSLE score
    """
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
    for train_idx, valid_idx in split.split(X, months):
        X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
        model = CatBoostRegressor(random_state=42,
                                 thread_count=4,
                                 verbose=False,
                                 loss_function='RMSE',
                                 eval_metric='RMSE',
                                 **params)
        model.fit(X_train, y_train,
                  verbose=False,
                  cat_features=["working_hours", "is_weekend", "maximum_hours"])
        oof_preds = np.expm1(model.predict(X_valid))
        oof_preds[oof_preds < 0] = 0
        oof_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), oof_preds))
        
    return oof_rmsle

In [ ]:
def train_without_folds(X, y, X_test, params):
    """
    Trains one model on a given dataset and returns its test data predictions and eature importances
    """
    
    model = CatBoostRegressor(random_state=42,
                             thread_count=4,
                             verbose=False,
                             loss_function='RMSE',
                             eval_metric='RMSE',
                             **params,
                            iterations=10)
    model.fit(X, y,
              verbose=False,
              cat_features=["working_hours", "is_weekend", "maximum_hours"])
    model_preds = np.expm1(model.predict(X_test))
    model_fi = model.feature_importances_
        
    return model_preds, model_fi

In [ ]:
def train_with_folds(X, y, X_test, months, params):
    """
    The function splits given train data into 10 folds and trains each model on each fold.
    Each model makes test predictions. Mean test predictions as vell as valid data predictions
    and score and feature importances are returned.
    """
    
    
    splits = 10
    skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
    oof_preds = np.zeros((X.shape[0],))
    model_preds = 0
    model_fi = 0
    for num, (train_idx, valid_idx) in enumerate(skf.split(X, months)):
        X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
        model = CatBoostRegressor(random_state=42,
                                 thread_count=4,
                                 verbose=False,
                                 loss_function='RMSE',
                                 eval_metric='RMSE',
                                 od_type="Iter",
                                 early_stopping_rounds=500,
                                 use_best_model=True,
    #                                      iterations=10000,
                                 **params)
        model.fit(X_train, y_train,
                  eval_set=(X_valid, y_valid),
                  verbose=False,
                  cat_features=["working_hours", "is_weekend", "maximum_hours"])
        model_preds += np.expm1(model.predict(X_test)) / splits
        model_fi += model.feature_importances_
        oof_preds[valid_idx] = np.expm1(model.predict(X_valid))
        oof_preds[oof_preds < 0] = 0
        print(f"Fold {num} RMSLE: {np.sqrt(mean_squared_log_error(np.expm1(y_valid), oof_preds[valid_idx]))}")
    #         print(f"Trees: {model.tree_count_}")
    model_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), oof_preds))
    
    
    return model_rmsle, model_preds, model_fi, oof_preds

In [ ]:
def get_blender_preds(X, X_test, y, months, params):
    """
    Splits the given dataset of base model predictions into folds and trains
    blender models. Test data predictions are returned.
    """
    
    splits = 10
    skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
    oof_preds = np.zeros((X.shape[0],))
    model_preds = 0
    for num, (train_idx, valid_idx) in enumerate(skf.split(X, months)):
        X_train, X_valid = np.ascontiguousarray(X.loc[train_idx]), np.ascontiguousarray(X.loc[valid_idx])
        y_train, y_valid = np.ascontiguousarray(y.loc[train_idx]), np.ascontiguousarray(y.loc[valid_idx])    
        
        blender = XGBRegressor(**params, verbosity=0)
        blender.fit(X_train, y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    early_stopping_rounds=100,
                    verbose=0)
        model_preds += np.expm1(blender.predict(np.ascontiguousarray(X_test))) / splits
        oof_preds[valid_idx] = np.expm1(blender.predict(X_valid))
        oof_preds[oof_preds < 0] = 0
        print(f"Blender fold {num} RMSLE: {np.sqrt(mean_squared_log_error(np.expm1(y_valid), oof_preds[valid_idx]))}")
    blender_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), oof_preds))
    print(f"Blender oof RMSLE is {blender_rmsle}")
    
    return model_preds, oof_preds

In [ ]:
def train_model_optuna(trial, X_train, X_valid, y_train, y_valid):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna. 
    Log loss of validation data predictions is returned to estimate hyperparameters effectiveness.
    """
    preds = 0
    
       
    #A set of hyperparameters to optimize by optuna
    #First set deep model
#     xgb_params = {
#                  "n_estimators": trial.suggest_categorical('n_estimators', [2000]),
#                  "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.2),
#                  "booster": trial.suggest_categorical('booster', ["gblinear"]),
#                  "reg_lambda": trial.suggest_float('reg_lambda', 0.00001, 0.9),
#                  "reg_alpha": trial.suggest_float('reg_alpha', 0.00001, 0.9),
#                  "base_score": trial.suggest_float('base_score', 0.1, 0.9),
#                  "random_state": trial.suggest_categorical('random_state', [42]),
#                  "n_jobs": trial.suggest_categorical('n_jobs', [4]),
#                     }

    xgb_params = {
                 "n_estimators": trial.suggest_categorical('n_estimators', [4000]),
                 "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.8),
                 "max_depth": trial.suggest_int("max_depth", 2, 30),
                 "booster": trial.suggest_categorical('booster', ["gbtree"]),
                 "tree_method": trial.suggest_categorical('tree_method', ["auto"]),
        
                 "reg_lambda": trial.suggest_float('reg_lambda', 0.00001, 0.9),
                 "reg_alpha": trial.suggest_float('reg_alpha', 0.00001, 0.9),
                 "random_state": trial.suggest_categorical('random_state', [42]),
                 "n_jobs": trial.suggest_categorical('n_jobs', [4]),
                    }


    model = XGBRegressor(**xgb_params)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="rmse",
              early_stopping_rounds=100,
              verbose=False)

    oof = np.expm1(model.predict(X_valid))
    print(f"Number of boosting rounds: {model.best_iteration}")
    
    return np.sqrt(mean_squared_log_error(np.expm1(y_valid), oof))

The code below runs hyperparameters optimization. It is commeted to save runtime.

In [ ]:
def optimize_params(X, y, months):
    # Splitting data into train and valid folds using target bins for stratification
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_idx, valid_idx in split.split(X, months):
        X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
        
    # Setting optuna verbosity to show only warning messages
    # If the line is uncommeted each iteration results will be shown
    # optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: train_model_optuna(trial, X_train, X_valid,
                                                        y_train, y_valid),
                   n_trials = 500)

    # Showing optimization results
    print('Number of finished trials:', len(study.trials))
    print('Best trial parameters:', study.best_trial.params)
    print('Best score:', study.best_value)

In [ ]:
%%time

# A list of feature importances
all_fi = []

# Initializing and filling predictions dataframe with datetime values
preds = pd.DataFrame()
preds["date_time"] = test_copy["date_time"].copy()

# The months will be used for folds split
months = train_copy.drop(7110, axis=0)["date_time"].dt.month

total_mean_rmsle = 0

X = pd.DataFrame()

train_preds_df = pd.DataFrame(index=np.arange(7110))
test_preds_df = pd.DataFrame(index=test.index)

for i, target in enumerate(targets):
# for i in [2]:
    target = targets[i]
    print(f"\nTraining for {target}...")
    
    group_fi = 0
    # Getting dataset for a current target in case there is need for different feature set per target
    X, X_test, y = prepare_dataset(pd.concat([train_copy, train_preds_df], axis=1), pd.concat([test_copy, test_preds_df], axis=1), i)
    oof_preds_for_blender = pd.DataFrame()
    test_preds_for_blender = pd.DataFrame()
    
    # Train base models with folds
    for num, params in enumerate(cb_params[i]):
        print(f"Training with this params: {params}")
        model_rmsle, model_preds, model_fi, oof_preds = train_with_folds(X, y[target], X_test, months, params)
        print(f"\nOverall {target} RMSLE: {model_rmsle}")
        total_mean_rmsle += model_rmsle / (len(cb_params[i]) * len(targets))
        all_fi.append(dict(zip(X_test.columns, model_fi)))
        oof_preds_for_blender[str(num)] = oof_preds
        test_preds_for_blender[str(num)] = model_preds
    display(oof_preds_for_blender)
#     optimize_params(oof_preds_for_blender, y[target], months)
    preds[target], blender_oof_preds = get_blender_preds(oof_preds_for_blender, test_preds_for_blender, y[target], months, xgb_params[i])
    train_preds_df[target+"_preds"] = np.log1p(blender_oof_preds)
    test_preds_df[target+"_preds"] = np.log1p(preds[target])
    
    
#     # Train base models without folds
#     for num, params in enumerate(cb_params[i]):
#         print(f"Training with this params: {params}")
#         model_rmsle = get_oof_rmsle(X, y[target], params, months)
#         print(f"Model's RMSLE is {model_rmsle}")
#         total_mean_rmsle += model_rmsle / (len(cb_params[i]) * len(targets))
        
#         model_preds, model_fi = train_without_folds(X, y[target], X_test, params)
#         group_fi += model_fi / len(cb_params[i])
        
# #         preds_for_blender[str(i)+"_"+str(num)] = model_preds
#         preds[target] += model_preds / len(cb_params[i])
#     all_fi.append(dict(zip(X_test.columns, group_fi)))

print(f"\n\nTotal RMSLE is {total_mean_rmsle}\n")

# **Feature importances**

In [ ]:
# Creating feature list from feature importance dictionaries
feature_list = set()
for i in np.arange(len(all_fi)):
    feature_list = set.union(feature_list, set(all_fi[i].keys()))
print(f"There are {len(feature_list)} unique features used for training: {feature_list}")

In [ ]:
# Combining feature importances of different models into one dataframe
df = pd.DataFrame(columns=["Feature"])
df["Feature"] = list(feature_list)
for i in np.arange(len(all_fi)):
    for key in all_fi[i].keys():
        df.loc[df["Feature"] == key, "Importance_" + str(i+1)] = all_fi[i][key] / 1000
df.fillna(0, inplace=True)
df["Overall_importance"] = df["Importance_1"] + df["Importance_2"] + df["Importance_3"]
df.sort_values("Overall_importance", ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
x = np.arange(0, len(df["Feature"]))
height = 0.3

fig, ax = plt.subplots(figsize=(12, 75))
bars1 = ax.barh(x-height, df["Importance_1"], height=height,
                color="cornflowerblue",
                edgecolor="black",
                label=target_names[0])
bars2 = ax.barh(x, df["Importance_2"], height=height,
                color="palevioletred",
                edgecolor="black",
                label=target_names[1])
bars3 = ax.barh(x+height, df["Importance_3"], height=height,
                color="mediumseagreen",
                edgecolor="black",
                label=target_names[2])
ax.set_title("Feature importances", fontsize=20, pad=5)
ax.set_ylabel("Feature names", fontsize=15, labelpad=5)
ax.set_xlabel("Feature importance", fontsize=15, labelpad=5)
ax.set_yticks(x)
ax.set_yticklabels(df["Feature"], fontsize=12)
# ax.set_xlim(0, 0.25)
# ax.set_xticks(np.arange(0, 0.275, 0.025))
ax.tick_params(axis="x", labelsize=12)
ax.grid(axis="x")
# ax2 = ax.secondary_xaxis('top')
# ax2.set_xlabel("Feature importance", fontsize=20, labelpad=15)
# # ax2.set_xlim(0, 0.25)
# # ax2.set_xticks(np.arange(0, 0.275, 0.025))
# ax2.tick_params(axis="x", labelsize=15)
# ax.legend(legend_lines, ["Original features", "Custom features"], fontsize=15, loc=1, bbox_to_anchor=(0, 0, 1, 0.97))
ax.legend(fontsize=13, loc="lower right")
plt.margins(0.04, 0.01)
plt.gca().invert_yaxis()

# **Predictions analysis and submission**

In [ ]:
preds.to_csv('submission.csv', index=False)
preds.head()

Let's compare predictions with the closest months from the train datasets.

In [ ]:
targets = train_copy[["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"]]

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=3, figsize=(16, 8))
plt.set_cmap("Set2")
plt.subplots_adjust(hspace = 0.3)

for i, target in enumerate(targets.columns):
    axs[i].plot(np.arange(0, 744, 1), targets.loc[train["month"]==12, target], label="Train, 12th month")
    axs[i].plot(np.arange(0, 744, 1), preds.loc[preds["date_time"].dt.month==1, target],
                label="Test, 1th month")
    axs[i].set_title(target_names[i], fontsize=15)
    axs[i].legend(fontsize=13)

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=3, figsize=(16, 8))
plt.set_cmap("Set2")
plt.subplots_adjust(hspace = 0.3)

for i, target in enumerate(targets.columns):
    axs[i].plot(np.arange(0, 720, 1), targets.loc[train["month"]==11, target], label="Train, 11th month")
    axs[i].plot(np.arange(0, 744, 1), preds.loc[preds["date_time"].dt.month==1, target],
                label="Test, 1th month")
    axs[i].set_title(target_names[i], fontsize=15)
    axs[i].legend(fontsize=13)

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=3, figsize=(16, 8))
plt.set_cmap("Set2")
plt.subplots_adjust(hspace = 0.3)

for i, target in enumerate(targets.columns):
    axs[i].plot(np.arange(0, 598, 1), targets.loc[:597, target], label="Train, from 10.3 to 4.4")
    axs[i].plot(np.arange(0, 596, 1), preds.loc[1651: , target],
                label="Test, from 10.3 to 4.4")
    axs[i].set_title(target_names[i], fontsize=15)
    axs[i].legend(fontsize=13)

As you can see, the predictions are the closest to the training set in the overlapping months (from March 10 to April 4). 